# Markov Chains - O que são e aplicações à geração de texto

## O que é uma Markov Chain?

Pela definição da wikipédia, uma *Markov chain* é

> Um modelo estocástico que descreve uma sequência de possíveis eventos em que a probabilidade de cada evento depende apenas do estado alcançado no evento anterior.

Um exemplo disto é um jogo de sorte, em que dois jogadores jogam várias rondas sucessivas. Se quisermos calcular as probabilidades de após mais uma ronda o jogo ficar com um determinado resultado precisamos apenas de saber qual o resultado atual, em nada interessa como chegámos a este resultado. (ex: se após uma ronda o jogo estiver 3-2, a probabilidade de após uma outra ronda o resultado ficar 3-3 será sempre 0.5 e de ficar 4-2 será também 0.5, em nada interessa se antes da última ronda jogada o resultado era 3-1 ou 2-2).

Podemos então dizer matemáticamente que:

$$ P(X_{k+1} | X_k, X_{k-1}, ..., X_1) = P(X_{k+1} | X_k) $$

Ou seja, dada uma sequência de $k$ estados, ($X_1, X_2, ..., X_k$), a probabilidade de alcançarmos após mais um evento um determinado estado $X_{k+1}$ depende apenas do estado actual, $X_k$.



## Como podemos usá-las para gerar texto?

Vamos assumir que o processo de escrita também segue esta lógica, isto é, que a palavra que vamos escrever a seguir depende apenas das $n$ palavras escritas antes. Temos então de calcular para cada palavra qual a probabilidade de esta aparacer imediatamente a seguir a cada conjunto de $n$ palavras. Fazê-lo é muito simples, vamos pegar em texto e contar quantas vezes cada palavra aparece depois de cada conjunto de $n$ palavras. Depois, com esta informação, vamos simular o processo de escrita, começamos com um conjunto de $n$ palavras e baseando-nos nas probabilidades que calculámos geramos a próxima palavra, repetimos isto as vezes necessárias para gerar um texto com o tamanho que quisermos.

Claro que isto não é de todo um bom modelo daquilo que é o processo de escrita, é melhor que um macaco a bater num teclado, mas no fundo é mais ou menos equivalente a um analfabeto a fazer corte e colagem com as páginas de um livro. Mas já vamos ver que resultados obtemos.



## Fontes

#### Eça de Queirós
Por razões que me ultrapassam, o departamento de **Física** da Universidade de Coimbra tem no seu site a obra completa de Eça de Queiroz em formato html (http://figaro.fis.uc.pt/queiros/lista_obras.html).

Estes textos foram posteriormente convertidos para .txt, ficheiros esses que estão agora hospedados num Dataset do Kaggle (https://www.kaggle.com/leite0407/ea-de-queiroz/).

In [1]:
import random
import os

import nltk

## Implementação

Temos duas tarefas:
* Contar num dado texto quantas vezes cada palavra aparece depois de um dado conjunto de $n$ palavras e guardar esta informação;
* Gerar texto com base na informação obtida.

#### Markov.train()
Para tornar esta implementação o mais abrangente possível, vamos aceitar como input uma lista. Assim, podemos criar uma cadeia com base em qualquer sequência de coisas, não necessariamente palavras. Daí no código (e na explicação daqui para a frente) os elementos desta lista serem designados tokens.

Para armazenar a informação vamos usar um dict em que cada conjunto de $n$ tokens vai estar associado a uma lista que contém todas os tokens que no texto vimos aparecer depois desse conjunto. Se virmos um dado token aparecer mais do que uma vez depois de um conjunto, vamos simplesmente adicioná-lo mais do que uma vez a esta lista.

Iteramos então pela lista de tokens e guardamos a informação em *markov_dict*.

#### Markov.generate_next_token()
Dado um conjunto de $n$ tokens, retorna aleatoriamente um token da lista associada ao conjunto.

#### Markov.generate_sequence()
Dada um conjunto de $n$ tokens por onde começar a sequência, gera uma sequência de *length* tokens, invocando a cada passo *generate_next_token()*.

In [2]:
class Markov:
    
    def __init__(self, n=1):
        
        self.n = n
        self.markov_dict = {}
    
    def train(self, tokens):
        ''' Conta quantas vezes cada token aparece à frente de cada conjunto de n tokens'''

        # Itera sobre tokens
        for i in range(self.n, len(tokens)):
            
            # Tuple com últimos n tokens
            past_tokens = tuple(tokens[i - self.n : i])
            
            # Verifica se past_tokens já existe no markov_dict
            if past_tokens not in self.markov_dict.keys():
                # Se não, incializa-o como sendo uma lista cujo único elemento é o token atual
                self.markov_dict[past_tokens] = [tokens[i]]
                
            else:
                # Se sim, apende o token atual à lista
                self.markov_dict[past_tokens].append(tokens[i])
    
    
    def generate_next_token(self, past_tokens):
        ''' Escolhe aleatoriamente um token de markov_dict[token_set] '''
        
        random_index = random.randint(0, len(self.markov_dict[past_tokens])-1)
        
        return self.markov_dict[past_tokens][random_index]
    
    def generate_sequence(self, start_tuple='random', length=20):
        
        if start_tuple == 'random':
            start_tuple =  list(self.markov_dict.keys())[random.randint(0, len(self.markov_dict.keys()) - 1)]
        
        results = list(start_tuple)
        
        for i in range(self.n, length):
            
            results.append(self.generate_next_token(tuple(results[i - self.n : i])))
        
        return results     

## Aplicações

### Eça de Queirós
Vamos então testar o que implementámos. Vamos começar por criar algumas cadeias com base na Obra Completa de Eça de Queirós.

In [6]:
obras = os.listdir('Eca/txt/')

print(obras)

['cartas_familiares.txt', 'cronicas_de_londres.txt', 'a_reliquia.txt', 'os_maias.txt', 'o_mandarim.txt', 'a_capital.txt', 'prosas_barbaras.txt', 'uma_campanha_alegre_II.txt', 'a_cidade_e_as_serras.txt', 'o_crime_do_padre_amaro.txt', 'contos.txt', 'cartas_de_inglaterra.txt', 'as_minas_de_salomao.txt', 'o_misterio_da_estrada_de_sintra.txt', 'uma_campanha_alegre_I.txt', 'a_ilustre_casa_de_ramires.txt', 'ecos_de_paris.txt', 'alves_e_ca.txt', 'o_conde_de_abranhos.txt', 'ultimas_paginas.txt', 'o_primo_basilio.txt']


Temos a obra completa do Eça, cada livro num ficheiro txt. Vamos juntar tudo na mesma string e separar esta string em tokens.

Não digo que vamos separar a string em palavras porque um token tanto pode ser uma palavra como um elemento de pontuação.
Por exemplo:

In [11]:
nltk.word_tokenize('- Falhamos a vida, menino!')

['-', 'Falhamos', 'a', 'vida', ',', 'menino', '!']

A cadeia vai tratar um ponto ou uma virgula exatamente da mesma maneira que trata uma palavra, pelo que ao gerar texto também vai colocar pontuação.

Vamos começar por criar uma cadeia com $n = 1$. Para este caso, o texto é gerado tendo em consideração na escolha do próximo token apenas o token imediatamente anterior.

(PS - Falhamos a vida! é uma quote d'Os Maias)

In [14]:
eca_str = ''
for obra in obras:
    eca_str = eca_str + open('Eca/txt/' + obra).read()
    
eca_tokens = nltk.word_tokenize(eca_str)

eca_markov = Markov(1)

eca_markov.train(eca_tokens)

In [93]:
sequence = eca_markov.generate_sequence(tuple(['A']), 200)

print(' '.join(sequence))

A seda preta , uma coisa esborrachada , a sua fortuna , onde o jardim . — confirmou o Carvalho . — Quem vier.. , foi o melhor o meio da francesa ofertado ao ser à janela — perguntou : – ficasse gravado no mesmo com que nesse mesmo ajudara culpadamente a voz , vejam que faculdade - Esta ideia de luvas claras etilintantes de repes verde , como Alexandre chamou , cópias das casas de santos seus interesses do Pedrinho , ululando sinistramente , do Templo – Nas janelas alumiadas , as feições de graça será condenado em costa de jacinto . Uma desgraça ... Carlos , deu um café , em que na Igreja de alegria que estava embaixo na manga de morrer ... O que ele compra va sos ; poderiam acomodar-se , diante da cozinha , de espírito de caravanas . E nos dias que as festas de lhe curava todos pareciam querer deste livro de Israel está , lancei -me tão cheio de constituir a conter a derrota ? ... Além disso a margarida caírem , legitima ? ... - Pois olhe que possuía um momentona mão , não houve quem p

Não faz absolutamente sentido nenhum! Como a cadeia apenas toma em consideração o token imediatamente anterior, se isolarmos quaisquer 2 tokens contíguos, eles fazem sentido entre si, mas o token que vem a seguir a este par não faz necessariamente sentido com o primeiro token.

Vamos então tentar usar uma cadeia com $n = 3$, assim temos a certeza que quaisquer 4 tokens contiguos vão fazer sentido uns com os outros.

In [30]:
eca_markov_3 = Markov(3)

eca_markov_3.train(eca_tokens)

In [90]:
sequence = eca_markov_3.generate_sequence(tuple(['A', 'casa', 'que']), 200)

print(' '.join(sequence))

A casa que os Maias vieram habitar em Lisboa , ainda conservava os antigos hábitos severamente encerrados ; ou por imposição paternal do marido , a horas lôbregas . S. Exª foi duas vezes regente ; assinou proclamações , decretos , sentenças ; constituiu ministérios ; interveio na política do seu país como um livro de ciência , ninguém mais se ocupara de fazer uma defesa que ecoará em toda a sua dedicação . Ela deixara-se cair sobre uma cadeira ao chão com um pontapé e estendeu-se sobre a cama , ou ço em cadanoite , ora por caminhos claros que soavam como lajes , ora sob a ramaria do carvalho de S. Froalengo , abrigava no sábado mais negro de janeiro , metido num portal da Rua de S. Roque . Apanharam-no ao pé de um altar , bradou : — Caramba ! Isso é uma hecatombe ! - Você está ali como um Deus antigo . Cravaste-lhe no seio as sete dores . Coube-lhe a dor e o escárnio . É necessário nesta vida um bocado de azul lavado e lustroso ; e a pobre velha à porta , numa efusão de simpatia : - Or

O problema de se usar um $n$ muito grande é que as vezes que uma expressão com tamanho $n$ aparece no texto diminui exponencialmente à medida que $n$ aumenta. Vamos analisar:

In [64]:
total = 0
count = 0
for key in eca_markov.markov_dict.keys():
    total += len(eca_markov.markov_dict[key])
    count += 1
    
print('Número médio de ocorrências de cada expressão de tamanho n = 1: ', total/count)

total = 0
count = 0
for key in eca_markov_3.markov_dict.keys():
    total += len(eca_markov_3.markov_dict[key])
    count += 1
    
print('Número médio de ocorrências de cada expressão de tamanho n = 3: ', total/count)

Número médio de ocorrências de cada expressão de tamanho n = 1:  43.55747770358087
Número médio de ocorrências de cada expressão de tamanho n = 3:  1.3482119386276605


Ou seja, em média, cada palavra (expressão de tamanho $n = 1$) aparece no texto 43.6 vezes, enquanto que cada expressão de tamaho $n = 3$ aparece em média 1.4 vezes.

Logo, ao gerarmos texto com a cadeia de $n = 3$ muitas vezes quando sorteamos a palavra seguinte só há uma hipótese, porque a combinação das últimas 3 palavras que vimos apenas aparece no texto 1 vez, pelo que, em vez de gerarmos novo texto, acabamos por quase citar o original.

Para comprovar isto, vamos experimentar construir uma cadeia com $n = 6$

In [65]:
eca_markov_6 = Markov(6)

eca_markov_6.train(eca_tokens)

In [68]:
sequence = eca_markov_6.generate_sequence(tuple(['A', 'casa', 'que', 'os', 'Maias', 'vieram']), 200)

' '.join(sequence)

'A casa que os Maias vieram habitar em Lisboa , no outono de 1875 , era conhecida na vizinhança da rua de S. Francisco de Paula , e em todo o bairro das Janelas Verdes , pela casa do Ramalhete ou simplesmente o Ramalhete . Apesar deste fresco nome de vivenda campestre , o Ramalhete , sombrio casarão de paredes severas , com um renque de estreitas varandas de ferro no primeiro andar , e por cima uma tímida fila de janelinhas abrigadas à beira do telhado , tinha o aspecto tristonho de Residência Eclesiástica que competia a uma edificação do reinado da Sr.ª D. Maria I : com uma sineta e com uma cruz no topo assimilar-se-ia a um Colégio de Jesuítas . O nome de Ramalhete provinha de certo dum revestimento quadrado de azulejos fazendo painel no lugar heráldico do Escudo de Armas , que nunca chegara a ser colocado , e representando um grande ramo de girassóis atado por uma fita onde se distinguiam letras e números duma data . Longos anos o Ramalhete permanecera desabitado , com teias de aranh

E o texto *'gerado'* é exatamente o famoso parágrafo inicial d'Os Maias!

Vamos agora experimentar uma coisa diferente. Em vez de construirmos a cadeia com base em palavras, vamos construí-la caractér a caractér. A cadeia vai funcionar exatamente da mesma maneira, só que em vez de escolher a próxima palavra com base nas $n$ anteriores, vai escolher o próximo *caractér* com base nos $n$ anteriores. Vamos começar com uma cadeia com $n = 1$ para ver o que acontece.

In [70]:
eca_letras = list(eca_str)

eca_letras_markov = Markov(1)

In [71]:
eca_letras_markov.train(eca_letras)

In [76]:
print(''.join(eca_letras_markov.generate_sequence(list('A'), 1000)))

Alho loro era oue 58 e broma da Não ás Estos He da ma. ria e – puma da ore Teplel sta Nãosói ja he tera b al, deziseleiarenão, co ro domamavo pa ujuresti osto tidmpo dra ranágortadantetrenses s pra.
Nil vaqu lto E Estumpo perarrcim coue rarrs, ra o feza Dinos cêsa. quanu pla os fe te, telvenhas codono jano rbo caltãoco obaqua nge, arês ouca Um heriarinãola Sa dalimema entamade, ntatas às a, Tico o liasmãoleimardus aronfare, sti lialiandasto!
s edas aina umpe, ogirrralto e nda s, atáramédelherado em da lgrfirare,
A lora elhe honde cossadadraro-lvauaspa pe Tr púvesconape as Prue murasis nco? Sa Não pevicentuina idos O ichefomolhos decasa tintues entumeizens, Barachiarona nfra de andencoamo. lhanapos, demugubras. ta endoroseixca cidamias, cuices ar esença à venuíto, s Cra ebratule ra ue e-lua amecoe MÍnça o rinaprdana co idos, émubra, démpasseico tradordo opo co. ficomo uertinana
	-lares mo pe mes m emo e a cher laguiãomus citi. rimo-desemevaveipe dinos e samorto laco à ve onantrgomam mem

Parece que a cadeia está a produzir coisas que até em alguns casos se parecem com palavras, há pelo menos uma sucessão de consoantes e vogais a formar sílabas, mas que não têm sentido absolutamente nenhum. Isto, claro, porque a cadeia, quando escolhe o próximo caractér, está apenas a ter em consideração o imediatamente anterior.

Vamos então tentar com outros valores de $n$

In [85]:
start_list = list('A casa que os Maias vieram')

for i in range(2, 20, 3):
    
    eca_letras_markov_i = Markov(i)
    eca_letras_markov_i.train(eca_letras)
    
    print('n = ', i, '\n')
    
    print(''.join(eca_letras_markov_i.generate_sequence(start_list[:i], 1500)), '\n\n\n')
    

n =  2 

A confir to à sapazito o desta! O que
pia-menstas mararo, suastária de que falto a de; e a das vesa, camem, quecente um fente apro, àquem Lubia, que a recesfacerrem nom re, acidame a e tão cal, aque a seu compor, pro, pé, ar arraçanua venspe, turosofeitame Cha, que Pros lacheiante cor um do va-ses,
que mentá? -SEADE num aus petanegas.
Olita dinevelasa, feiado quizerecuros:
- Na apa e oseus urinhomo retoceiamenrança". Coi Pôs mulha e o aque enhom só: mos aistónip! Adeque ou pindos parrenvese ta, com las quelêncietito delego excentam pazo, a pervo, dices mo exarassou carbalva em, socas condado! heio! — Maramembém?
Conós adadonde – Stei, pera erira trade atracorem arra em orde, que afé dendo, essuanta peraça sesor, infaliz repo!

— Pindo do se –
A rés visparedressolearande moita, marrespareganteirrionárias cam-sestronso: dúlito che sente se o ladomo de, os houva seias papros sos fazempedontens dormo, Galém-serourmaio, algorpricavamanteriamonte, idademas o bêbente fa, quem um essa

Para $n = 2$ temos neologismos como *visparedressolearande* ou *marrespareganteirrionárias*, nem o Eça se lembrava destes.

Para $n$ cada vez maior, o texto parece cada vez fazer mais sentido, mas mesmo assim no geral não tem lógica absolutamente nenhuma.

Começa a tornar-se claro que o texto gerado por Markov Chains não faz sentido absolutamente nenhum. É fácil compreender porquê: Ao gerarmos texto, basicamente estamos a juntar ao acaso vários segmentos de palavras ou caracteres. É como se déssemos um livro a um analfabeto e lhe disséssemos para recortar várias frases e depois colá-las pela ordem que bem entendesse, apenas tendo de obedecer à regra que a última e a primeira palavras de duas frases contíguas têm de coincidir, claro que os resultados não vão ser ótimos, mas pelo menos são engraçados.

### Corretor Eça

Então mas há alguma aplicação prática de tudo isto? Claro que sim! Os corretores automáticos dos telemóveis têm na sua base uma Markov Chain. Por isso é que há medida que se usa mais o teclado, o corretor automático vai ficando melhor, ele regista aquilo que já escreveste e usa exatamente esta ideia das Markov Chains para tentar prever qual a próxima palavra que vais escrever.

Então com base nesta ideia criei num pequeno script um *'corretor automático'* que usa como base a primeira Markov Chain que construímos aqui (palavras, $n = 1$).

https://repl.it/repls/StridentSteelblueDoom


## Conclusão

Usar Markov Chains para gerar texto não produz propriamente obras literárias brilhantes, mas de qualquer forma não deixa de ter a sua piada.

O que é interessante é que de certa forma o estilo de Eça de Queirós faz-se notar nos textos gerados, pelo que construir uma cadeia com base num determinado texto e gerar um pequeno texto com ela pode ser uma ferramenta útil para compreender melhor o texto em causa.